In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
import scipy.sparse as sp
file_path = '/content/drive/My Drive/266 Project/efcamdat_sub.csv'

df = pd.read_csv(file_path)

print(df.head())

       id  level  unit  learner_id learner_nationality  grade  \
0  679604      7     3      114335                  br     90   
1  151196      9     2      136139                  sa     94   
2  117084      9     4       34715                  br     88   
3  113857      7     6       90269                  fr     90   
4   22083      9     3       48465                  br     94   

                      date  topic_id  \
0  2013-11-09 20:50:31.707        51   
1  2012-09-25 06:01:08.117        66   
2  2011-08-28 08:01:15.677        68   
3  2011-07-31 14:51:22.547        54   
4  2011-08-31 16:41:04.210        67   

                                                text  cefr_numeric  \
0  From:l AS xxx@hotmail.com To: AS xxx@IXW.corpo...             3   
1  I am so glad to receive this email from you. A...             3   
2  Hi Fun Skydive, so I give up of my idea. I und...             3   
3  Dear James, Some serious problems have been br...             3   
4  Dear Sue, Thank

**Check data**

In [3]:
df.columns

Index(['id', 'level', 'unit', 'learner_id', 'learner_nationality', 'grade',
       'date', 'topic_id', 'text', 'cefr_numeric', 'cefr_grouped'],
      dtype='object')

In [4]:
df.shape[0]

377967

In [5]:
text = df['text']
labels = df['cefr_numeric']

In [6]:
(train_texts, test_texts, train_labels, test_labels) = train_test_split(text, labels, test_size=.1, shuffle=True)

In [7]:
len(test_texts)

37797

**Divide the data into training, validation, and test sets**

In [8]:
# Use a smaller dataset for faster experimentation
valid_texts_s = train_texts.iloc[:1000]
valid_labels_s = train_labels.iloc[:1000]
train_texts_s = train_texts.iloc[1000:11000]
train_labels_s = train_labels.iloc[1000:11000]
test_texts_s = test_texts.iloc[:1000]
test_labels_s = test_labels.iloc[:1000]

In [9]:
# Define model checkpoint and tokenizer
model_ckpt = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
base_model = TFAutoModel.from_pretrained(model_ckpt)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [10]:
# Tokenize data
max_length = 200
train_encodings = tokenizer(list(train_texts_s), truncation=True, padding=True, max_length=max_length, return_tensors='tf')
valid_encodings = tokenizer(list(valid_texts_s), truncation=True, padding=True, max_length=max_length, return_tensors='tf')
test_encodings = tokenizer(list(test_texts_s), truncation=True, padding=True, max_length=max_length, return_tensors='tf')

**Model architecture with GRU**

In [13]:
# Define the model architecture with GRU
def create_roberta_gru_model(max_length, num_classes, dropout=0.3, learning_rate=0.0001):
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

    # RoBERTa base model
    roberta_output = base_model(input_ids=input_ids, attention_mask=attention_mask)[0]

    # GRU layer
    gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=256, return_sequences=False))(roberta_output)
    gru = tf.keras.layers.Dropout(dropout)(gru)

    # Classification layer
    classification = tf.keras.layers.Dense(num_classes, activation='softmax', name='classification_layer')(gru)

    # Define the model
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[classification])

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=0.1)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

    return model

In [14]:
# Prepare the model
df['labels'] = df['cefr_numeric'].apply(lambda x: x-1)
num_classes = len(df['labels'].unique())
model = create_roberta_gru_model(max_length, num_classes)

In [15]:
# Prepare datasets for training and validation
train_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": train_encodings.input_ids, "attention_mask": train_encodings.attention_mask}, train_labels_s))
train_dataset = train_dataset.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": valid_encodings.input_ids, "attention_mask": valid_encodings.attention_mask}, valid_labels_s))
valid_dataset = valid_dataset.batch(32).prefetch(tf.data.experimental.AUTOTUNE)


In [16]:
# Clear session and free memory before starting the training
tf.keras.backend.clear_session()
import gc
gc.collect()


52436

**Various features from the text - Tokenize the Text Data**

In [19]:

model_name = "intfloat/multilingual-e5-large-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_length = 200

train_encodings = tokenizer(list(train_texts_s), truncation=True, padding=True, max_length=max_length, return_tensors='tf')
valid_encodings = tokenizer(list(valid_texts_s), truncation=True, padding=True, max_length=max_length, return_tensors='tf')
test_encodings = tokenizer(list(test_texts_s), truncation=True, padding=True, max_length=max_length, return_tensors='tf')


tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

**Transform categorical metadata into numerical format**

In [20]:
df['learner_id_categorical'] = df['learner_id'].astype('category').cat.codes
df['topic_id_categorical'] = df['topic_id'].astype('category').cat.codes

# Select relevant features (only available ones)
available_columns = ['learner_id_categorical', 'cefr_numeric']
additional_features = df[available_columns]

# Standardize additional features (optional)
scaler = StandardScaler()
additional_features_scaled = scaler.fit_transform(additional_features)

# Split additional features
train_additional_features = additional_features_scaled[:len(train_texts_s)]
valid_additional_features = additional_features_scaled[len(train_texts_s):len(train_texts_s) + len(valid_texts_s)]
test_additional_features = additional_features_scaled[len(train_texts_s) + len(valid_texts_s):len(train_texts_s) + len(valid_texts_s) + len(test_texts_s)]

# Print shapes to ensure they match
print("Shapes of feature sets:")
print(f"train_encodings.input_ids: {train_encodings.input_ids.shape}")
print(f"train_additional_features: {train_additional_features.shape}")


print(f"valid_encodings.input_ids: {valid_encodings.input_ids.shape}")
print(f"valid_additional_features: {valid_additional_features.shape}")


print(f"test_encodings.input_ids: {test_encodings.input_ids.shape}")
print(f"test_additional_features: {test_additional_features.shape}")


Shapes of feature sets:
train_encodings.input_ids: (10000, 200)
train_additional_features: (10000, 2)
valid_encodings.input_ids: (1000, 200)
valid_additional_features: (1000, 2)
test_encodings.input_ids: (1000, 200)
test_additional_features: (1000, 2)


**Generate TF-IDF Features - Convert the text into TF-IDF features**

In [21]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Use N-grams (1,2) for TF-IDF

train_tfidf = tfidf_vectorizer.fit_transform(train_texts_s)
valid_tfidf = tfidf_vectorizer.transform(valid_texts_s)
test_tfidf = tfidf_vectorizer.transform(test_texts_s)

**Ensure that the number of samples (rows) in each feature set matches**

In [22]:
assert train_encodings.input_ids.shape[0] == train_additional_features.shape[0] == train_tfidf.shape[0]
assert valid_encodings.input_ids.shape[0] == valid_additional_features.shape[0] == valid_tfidf.shape[0]
assert test_encodings.input_ids.shape[0] == test_additional_features.shape[0] == test_tfidf.shape[0]


**Merge the different feature sets into a single matrix for each data split**

In [23]:
train_combined_features = sp.hstack([sp.csr_matrix(train_encodings.input_ids.numpy()), sp.csr_matrix(train_additional_features), train_tfidf])
valid_combined_features = sp.hstack([sp.csr_matrix(valid_encodings.input_ids.numpy()), sp.csr_matrix(valid_additional_features), valid_tfidf])
test_combined_features = sp.hstack([sp.csr_matrix(test_encodings.input_ids.numpy()), sp.csr_matrix(test_additional_features), test_tfidf])

**Standardize the combined feature sets to have mean 0 and variance 1**

In [24]:
scaler = StandardScaler(with_mean=False)
train_combined_features = scaler.fit_transform(train_combined_features)
valid_combined_features = scaler.transform(valid_combined_features)
test_combined_features = scaler.transform(test_combined_features)

**Train a logistic regression model using the combined features**

In [25]:
model = LogisticRegression(max_iter=1000)
model.fit(train_combined_features, train_labels_s)

# Predict on the test set
y_pred = model.predict(test_combined_features)

# Evaluate the model
print(classification_report(test_labels_s, y_pred))

              precision    recall  f1-score   support

           1       0.93      0.96      0.95       278
           2       0.92      0.92      0.92       253
           3       0.85      0.97      0.91       269
           4       0.91      0.81      0.86       161
           5       1.00      0.24      0.39        33
           6       1.00      0.17      0.29         6

    accuracy                           0.90      1000
   macro avg       0.94      0.68      0.72      1000
weighted avg       0.91      0.90      0.89      1000



- Precision: The ratio of correctly predicted positive and might be the - classifier not to label as positive a sample that is negative.
- Recall (Sensitivity): The ratio of correctly predicted positive so the classifier to find all the positive samples.
- F1-Score: It is a good measure of the classifier’s performance when we need a balance between precision and recall.
- Support = The number of actual


Classes 1, 2, 3 and 4 have many examples, so the model does well in predicting them.

Classes 5, and 6 have fewer examples, making it harder for the model to learn and predict them accurately.



**Verify unique labels in the training set**

In [26]:
unique_labels = train_labels_s.unique()
print(f"Unique labels in training set: {unique_labels}")

# Verify unique labels in the test predictions
unique_predictions = set(y_pred)
print(f"Unique predictions: {unique_predictions}")


Unique labels in training set: [2 3 1 4 5 6]
Unique predictions: {1, 2, 3, 4, 5, 6}


**Define rubrics for different proficiency levels**

In [27]:
rubrics = {
    '1': {
        'Grammar': 'Basic sentence structures with frequent grammatical errors.',
        'Vocabulary': 'Limited vocabulary with repetitive word use.',
        'Sentence Structure': 'Simple and repetitive sentence structures.',
        'Coherence': 'Limited coherence with ideas not clearly connected.'
    },
    '2': {
        'Grammar': 'More complex sentences with occasional grammatical errors.',
        'Vocabulary': 'Adequate vocabulary with some variety and appropriate usage.',
        'Sentence Structure': 'Mix of simple and compound sentences with some complexity.',
        'Coherence': 'Generally coherent with some clear connections between ideas.'
    },
    '3': {
        'Grammar': 'Complex and varied sentence structures with minimal grammatical errors.',
        'Vocabulary': 'Extensive and varied vocabulary with precise and contextually appropriate usage.',
        'Sentence Structure': 'Varied and complex sentence structures that enhance clarity and style.',
        'Coherence': 'Highly coherent with well-organized and logically connected ideas.'
    },
    '4': {
        'Grammar': 'Advanced use of complex sentence structures with rare grammatical errors.',
        'Vocabulary': 'Rich and varied vocabulary with contextually accurate and sophisticated usage.',
        'Sentence Structure': 'Highly complex and varied sentence structures enhancing readability and style.',
        'Coherence': 'Exceptionally coherent with well-organized, logically connected ideas and smooth transitions.'
    },
    '5': {
        'Grammar': 'Near-perfect use of grammar with exceptional sentence structures.',
        'Vocabulary': 'Extremely rich and diverse vocabulary with precise and nuanced usage.',
        'Sentence Structure': 'Masterful use of varied and complex sentence structures.',
        'Coherence': 'Outstanding coherence with seamlessly connected ideas and flawless transitions.'
    },
    '6': {
        'Grammar': 'Exceptional grammar usage with no grammatical errors.',
        'Vocabulary': 'Exceptional vocabulary usage with a wide range of words used accurately.',
        'Sentence Structure': 'Exceptional sentence structures with varied and complex forms.',
        'Coherence': 'Perfect coherence with fully integrated ideas and flawless transitions.'
    }
}


**Generate feedback for each text based on the predicted proficiency level**

In [29]:
def provide_feedback(predictions, rubrics):
    feedback = []
    for pred in predictions:
        level = str(pred)
        feedback.append(rubrics.get(level, {
            'Grammar': 'No rubric available for this level.',
            'Vocabulary': 'No rubric available for this level.',
            'Sentence Structure': 'No rubric available for this level.',
            'Coherence': 'No rubric available for this level.'
        }))
    return feedback

feedback = provide_feedback(y_pred, rubrics)

for i in range(5):
    print(f"Text: {test_texts_s.iloc[i]}")
    print(f"Predicted Level: {y_pred[i]}")
    print(f"Feedback: {feedback[i]}")
    print()


Text: I just D heard the melody WC song &quot; home C Home improvement C Improvement &quot; by josh Woodward three times WC thrice, and in my opinion PU , the tune's quite sticky, so PR with not much pulse MW in it , PU ; the singer has a high register WC range . The singer can hit some really high notes , PU ; you really can dance to it , PU . I like how the mood changes. First, it's edgy, and then it's romantic. I like how it changes the mood. First, it's edgy PU , and then MW it is romantic. Finally PU , the song has kind of a sentimental melody. Although this type of music is not to D my preference. I recommend it because it is not offensive. So if you want to be relaxed PU , you can hear it.
Predicted Level: 3
Feedback: {'Grammar': 'Complex and varied sentence structures with minimal grammatical errors.', 'Vocabulary': 'Extensive and varied vocabulary with precise and contextually appropriate usage.', 'Sentence Structure': 'Varied and complex sentence structures that enhance clari